I redid the raw data and need to re-cater the function to that.
Need to also add outputs for gender and pell

In [2]:
import pandas as pd
import numpy as np

Notes and explanations:
This function outputs 4 dataframes in respective order:

    - df1: the overall totals for all non-high school student transfers by FY/term

    - df2: the race, transfer count/rate, and group size by FY/term

    - df3: the firstgen status, transfer count/rate, and group size by FY/term

    - df4: each ind student's tech_id, transfer status, graduation status, cohort year/term, admission status, first gen status, and race


This function uses data from the below tables:

    - ISRS.ST_COU: the last semester a student attended RCC

    - ISRS.ST_TERM_MGMT: the admission status of the a student during their final RCC semester

    - APPSODS.V_STUDENT_PERSISTENCE_DTL2: each student's final record of retention, transfer, and graduation.

    - APPSODS.V_IPEDS_ETHNIC_RACE: what a students ethnicity/race is
    
    - APPSODS.V_FIRST_GENERATION: whether or not a student is first generation in their family to go to college

In [3]:
##retrieve data
persDTL2_10 = pd.read_csv('persDTL2_10.csv') #df1   now includes first gen, pell, and gender
courseRecords_10 = pd.read_csv('courseRecords_10.csv')
transferRecords_10 = pd.read_csv('transferRecords_10.csv') #df4
race_10 = pd.read_csv('race_10.csv') #df3
admstat_10 = pd.read_csv('admstat_10.csv') 
last = pd.read_csv('lastsemester_10yrs.csv') #df2

In [22]:
persDTL2_10

,TECH_ID,COHORT_ENTERING_YEAR_TERM_CODE,FY,Semester,FIRST_GENERATION_STATUS_MN,FIRST_GENERATION_STATUS_FED,PELL_ELIGIBLE,GENDER,FIRST_FALL_RETAINED,FIRST_FALL_TRANSFERRED,...,EIGTH_FALL_RETAINED,EIGTH_FALL_TRANSFERRED,EIGTH_FALL_GRADUATED,EIGTH_FALL_TRANS_GRADUATED,EIGTH_FALL_SUCCESS,EIGTH_SPRING_RETAINED,EIGTH_SPRING_TRANSFERRED,EIGTH_SPRING_GRADUATED,EIGTH_SPRING_TRANS_GRADUATED,EIGTH_SPRING_SUCCESS
0,1185,20203,2020,3,No,Yes,Yes,Male,1,0,...,0,0,0,0,0,0,0,0,0,0
1,9722,20205,2020,5,No,Yes,Yes,Female,1,0,...,0,0,0,0,0,0,0,0,0,0
2,9929,20143,2014,3,Yes,Yes,Yes,Female,1,0,...,0,0,1,0,1,0,0,1,0,1
3,11387,20133,2013,3,Unknown,Unknown,Unknown,Male,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12943,20143,2014,3,No,Yes,No,Female,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,16532959,20243,2024,3,No,Yes,No,Female,0,0,...,0,0,0,0,0,0,0,0,0,0
9800,16534331,20243,2024,3,Unknown,Unknown,Yes,Female,0,0,...,0,0,0,0,0,0,0,0,0,0
9801,16539311,20243,2024,3,Unknown,Unknown,Unknown,Unknown,0,0,...,0,0,0,0,0,0,0,0,0,0
9802,16539765,20243,2024,3,No,No,Unknown,Male,0,0,...,0,0,0,0,0,0,0,0,0,0


What needs to happen?

    - merge the graduation and transfer YRTR
        if GRADUATE is Y then the transfer_YRTR is a graduation YRTR
    - merge with 'last' df
        remove any transfer_yrtr that is not greater than the last semester at RCC YRTR
    - drop any unneccessary columns ENROLLMENT BEGIN AND END
    - create 2 dfs from what you currently have
        one will only have the 'youngest' transfer YRTR to get the college they went to directly after RCC
        one will only have rows with a 'Y' in GRADUATE
    

What ouput do I need?
    - df_out1:
        - name of college transferring to
        - all-time transfer tally
        - all-time "completed" cohorts transfer tally by: race, pell, firstgen, gender
    - df_out2: 
        - name of college transferring to
        - last 5 "completed" cohorts transfer tally
        - last 5 "completed" cohorts transfer tally by: race, pell, firstgen, gender
    - df_out3: 
        - name of college transferring to
        - last 1 "completed" cohorts transfer tally
        - last 1 "completed" cohorts transfer tally by: race, pell, firstgen, gender
    - df_out4:
        - name of college
        - college information
        - graduation tally by FY cohort
        - transfer tally by FY cohort
    - correlation checks for college_type, public_private, graduate, enrollment_status, gender, firstgen, race, pell

In [4]:
def collegeTransfers(df1, df2, df3, df4):
    #make all your copies
    persDTL_2 = df1.copy()
    last2 = df2.copy()
    race2 = df3.copy()
    tr10 = df4.copy()
    #make your output df
    colleges = pd.DataFrame(columns=['College','Total','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_currentFY = pd.DataFrame(columns=['College','FY2024','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_lastCompleteCohort = pd.DataFrame(columns=['College','FY2020','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_last5CompleteCohorts = pd.DataFrame(columns=['College','FY2016','FY2017','FY2018','FY2019','FY2020','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    #slice to get only what you need
    persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
    persDTL2_slice = persDTL_2[['TECH_ID','FY','Semester','FIRST_GENERATION_STATUS_MN','PELL_ELIGIBLE','GENDER','transferred_at','COHORT_ENTERING_YEAR_TERM_CODE']]
    slice1 = persDTL2_slice.loc[persDTL2_slice['transferred_at'] > 0,:]
    last_slice = last2.loc[last['YRTR']>20125,:]
    tr10['TRANSFER_YRTR'].fillna(tr10['GRADUATION_YRTR'], inplace=True)
    tr10_2 = tr10.drop(columns=['GRADUATION_YRTR'])
    ##merge all the slices together before working with the transfer records
    glued1 = pd.merge(slice1, last_slice, how='left',on='TECH_ID')
    glued2 = pd.merge(glued1, race2, how='left',on='TECH_ID')
    #merge with transfer records
    mass = pd.merge(glued2, tr10_2, how='inner',on='TECH_ID')
    mass2 = mass.loc[mass['TRANSFER_YRTR'] > mass['COHORT_ENTERING_YEAR_TERM_CODE'],['TECH_ID','COLLEGE_NAME','COLLEGE_STATE','COLLEGE_TYPE','TRANSFER_YRTR','COHORT_ENTERING_YEAR_TERM_CODE','GRADUATE','PUBLIC_PRIVATE','ENROLLMENT_STATUS']]
    #######################################
    ##not sure if we will use this df yet
    grads = mass2.loc[mass2['GRADUATE']=='Y',:]
    #######################################
    #first college after RCC df
    nonRCC = mass2.loc[(mass2['COLLEGE_NAME'] != 'RIVERLAND COMMUNITY COLLEGE'),:]
    firstNew = nonRCC.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID','COLLEGE_NAME'], keep='first', ignore_index=True)
    firstNew2 = firstNew.sort_values(by=['COLLEGE_TYPE'], ascending=False)
    firstNew3 = firstNew2.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID'], keep='first', ignore_index=True)
    lastNew = nonRCC.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID','COLLEGE_NAME'], keep='last', ignore_index=True)
    lastNew2 = lastNew.sort_values(by=['GRADUATE'])
    lastNew3 = lastNew2.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID'], keep='last', ignore_index=True)
    ##df with student info, first college info and last college info
    lastFirstNew = pd.merge(firstNew3, lastNew3, how='left', on='TECH_ID', suffixes=('_FIRST','_LAST'))
    mass3 = pd.merge(glued2, lastFirstNew, on='TECH_ID', how='left')
    mass4 = mass3.drop(columns=['COHORT_ENTERING_YEAR_TERM_CODE_FIRST','COHORT_ENTERING_YEAR_TERM_CODE_LAST'])
    colls = mass4[['COLLEGE_NAME_FIRST','COLLEGE_TYPE_FIRST','COLLEGE_STATE_FIRST','PUBLIC_PRIVATE_FIRST']].drop_duplicates().reset_index()
    FY20 = mass4.loc[mass4['FY']==2020,:]
    last5 = mass4.loc[mass4['FY']>2015,:]
    for i in colls.index:
        col = colls.loc[i,'COLLEGE_NAME_FIRST']
        typ = colls.loc[i,'COLLEGE_TYPE_FIRST']
        state = colls.loc[i,'COLLEGE_STATE_FIRST']
        prp = colls.loc[i,'PUBLIC_PRIVATE_FIRST']
        college_allTime = pd.DataFrame(
            {'College':col,
            'Total':mass4.loc[mass4['COLLEGE_NAME_FIRST']==col,'TECH_ID'].count(),
            'Pell':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['COLLEGE_NAME_FIRST']==mass4['COLLEGE_NAME_LAST']) &(mass4['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges = pd.concat([colleges,college_allTime])
        colleges_lastCompleteCohort2 = pd.DataFrame(
            {'College':col,
            'FY2020':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'Pell':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['COLLEGE_NAME_FIRST']==FY20['COLLEGE_NAME_LAST']) &(FY20['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges_lastCompleteCohort = pd.concat([colleges_lastCompleteCohort,colleges_lastCompleteCohort2])

        colleges_last5CompleteCohorts2 = pd.DataFrame(
            {'College':col,
            'FY2016':last5.loc[(last5['FY']==2016) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2017':last5.loc[(last5['FY']==2017) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2018':last5.loc[(last5['FY']==2018) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2019':last5.loc[(last5['FY']==2019) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2020':last5.loc[(last5['FY']==2020) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'Pell':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['COLLEGE_NAME_FIRST']==last5['COLLEGE_NAME_LAST']) &(last5['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges_last5CompleteCohorts = pd.concat([colleges_last5CompleteCohorts,colleges_last5CompleteCohorts2])
    df_out1 = colleges.sort_values(by='Total', ascending=False)
    df_out2 = colleges_lastCompleteCohort.sort_values(by='FY2020', ascending=False)
    df_out3 = colleges_last5CompleteCohorts.sort_values(by='FY2016', ascending=False)
    df_out4 = mass4.drop(columns=['TECH_ID'])
    publicVSprivate_type_state = pd.DataFrame(
        {'Demographic':['ALL','ALL','Female','Female','Male','Male','Pell','FirstGen','Pell','FirstGen'],
        'FY':['ALL','2020','ALL','2020','ALL','2020','ALL','ALL','2020','2020'],
        'Public':[sum(colleges.loc[colleges['Pub_Priv']=='P','Total']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FY2020']),sum(colleges.loc[colleges['Pub_Priv']=='P','Female']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','Female']), sum(colleges.loc[colleges['Pub_Priv']=='P','Male']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','Male']), sum(colleges.loc[colleges['Pub_Priv']=='P','Pell']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FirstGen']), sum(colleges.loc[colleges['Pub_Priv']=='P','Pell']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FirstGen'])],
        'Private':[sum(colleges.loc[colleges['Pub_Priv']=='V','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FY2020']),sum(colleges.loc[colleges['Pub_Priv']=='V','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','Female']), sum(colleges.loc[colleges['Pub_Priv']=='V','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','Male']), sum(colleges.loc[colleges['Pub_Priv']=='V','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FirstGen']), sum(colleges.loc[colleges['Pub_Priv']=='V','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FirstGen'])],
        '_2yr':[sum(colleges.loc[colleges['College_type']=='2','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FY2020']), sum(colleges.loc[colleges['College_type']=='2','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','Female']), sum(colleges.loc[colleges['College_type']=='2','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','Male']), sum(colleges.loc[colleges['College_type']=='2','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FirstGen']), sum(colleges.loc[colleges['College_type']=='2','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FirstGen'])],
        '_4yr':[sum(colleges.loc[colleges['College_type']=='4','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FY2020']), sum(colleges.loc[colleges['College_type']=='4','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','Female']), sum(colleges.loc[colleges['College_type']=='4','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','Male']), sum(colleges.loc[colleges['College_type']=='4','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FirstGen']), sum(colleges.loc[colleges['College_type']=='4','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FirstGen'])],
        'MN':[sum(colleges.loc[colleges['State']=='MN','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FY2020']), sum(colleges.loc[colleges['State']=='MN','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','Female']), sum(colleges.loc[colleges['State']=='MN','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','Male']), sum(colleges.loc[colleges['State']=='MN','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FirstGen']), sum(colleges.loc[colleges['State']=='MN','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FirstGen'])],
        'WI':[sum(colleges.loc[colleges['State']=='WI','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FY2020']), sum(colleges.loc[colleges['State']=='WI','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','Female']), sum(colleges.loc[colleges['State']=='WI','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','Male']), sum(colleges.loc[colleges['State']=='WI','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FirstGen']), sum(colleges.loc[colleges['State']=='WI','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FirstGen'])],
        'IA':[sum(colleges.loc[colleges['State']=='IA','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FY2020']), sum(colleges.loc[colleges['State']=='IA','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','Female']), sum(colleges.loc[colleges['State']=='IA','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','Male']), sum(colleges.loc[colleges['State']=='IA','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FirstGen']), sum(colleges.loc[colleges['State']=='IA','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FirstGen'])],
        'IL':[sum(colleges.loc[colleges['State']=='IL','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FY2020']), sum(colleges.loc[colleges['State']=='IL','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','Female']), sum(colleges.loc[colleges['State']=='IL','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','Male']), sum(colleges.loc[colleges['State']=='IL','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FirstGen']), sum(colleges.loc[colleges['State']=='IL','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FirstGen'])],
        'ND':[sum(colleges.loc[colleges['State']=='ND','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FY2020']), sum(colleges.loc[colleges['State']=='ND','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','Female']), sum(colleges.loc[colleges['State']=='ND','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','Male']), sum(colleges.loc[colleges['State']=='ND','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FirstGen']), sum(colleges.loc[colleges['State']=='ND','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FirstGen'])],
        'SD':[sum(colleges.loc[colleges['State']=='SD','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FY2020']), sum(colleges.loc[colleges['State']=='SD','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','Female']), sum(colleges.loc[colleges['State']=='SD','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','Male']), sum(colleges.loc[colleges['State']=='SD','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FirstGen']), sum(colleges.loc[colleges['State']=='SD','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FirstGen'])],
        'Other':[sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Total']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Female']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Pell']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'FirstGen']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Pell']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'FirstGen'])]
        },
        index = [1,2,3,4,5,6,7,8,9,10])

    return df_out1, df_out2, df_out3, df_out4, publicVSprivate_type_state

In [5]:
Colleges_allTime, Colleges_lastCompleteCohort, Colleges_last5CompleteCohorts, allTransfers, publicVSprivate_type_state = collegeTransfers(persDTL2_10, last, race_10, transferRecords_10)


In [94]:
publicVSprivate_type_state

,Demographic,FY,Public,Private,_2yr,_4yr,MN,WI,IA,IL,ND,SD,Other
1,ALL,ALL,1261,340,848,750,1038,100,118,23,13,23,286
2,ALL,2020,112,35,71,76,99,5,6,4,2,3,11
3,Female,ALL,665,210,481,392,624,44,64,8,5,11,167
4,Female,2020,66,26,46,46,65,3,5,2,0,0,17
5,Male,ALL,594,130,366,357,412,56,54,15,8,12,167
6,Male,2020,45,9,25,29,33,2,1,2,2,3,11
7,Pell,ALL,661,215,478,397,547,43,78,14,7,8,179
8,FirstGen,ALL,32,16,22,26,32,3,0,1,1,2,9
9,Pell,2020,661,215,478,397,547,43,78,14,7,8,179
10,FirstGen,2020,32,16,22,26,32,3,0,1,1,2,9


In [6]:
Colleges_allTime.to_csv('Colleges_allTime.csv', index=False)
Colleges_lastCompleteCohort.to_csv('Colleges_lastCompleteCohort.csv', index=False)
Colleges_last5CompleteCohorts.to_csv('Colleges_last5CompleteCohorts.csv', index=False)
allTransfers.to_csv('allTransfers.csv', index=False)
publicVSprivate_type_state.to_csv('publicVSprivate_type_state.csv', index=False)
